In [0]:
!ls

deceptive-opinion.csv		   drive     sample_data
deceptive-opinion-spam-corpus.zip  model.h5


In [0]:
from google.colab import files
uploaded = files.upload()

Saving deceptive-opinion-spam-corpus.zip to deceptive-opinion-spam-corpus.zip


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/My\ Drive/deceptive-opinion-spam-corpus.zip .
# !cp drive/My\ Drive/glove.6B.200d.txt .

In [0]:
!unzip deceptive-opinion-spam-corpus.zip

Archive:  deceptive-opinion-spam-corpus.zip
  inflating: deceptive-opinion.csv   


In [0]:
!pip3 install --quiet sentencepiece

    100% |████████████████████████████████| 1.0MB 18.8MB/s 


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, BatchNormalization, Dropout, Reshape
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import keras
from sklearn.utils import resample

from keras.layers import Activation, Conv1D, Conv2D, Bidirectional, GlobalMaxPool1D
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, MaxPooling1D
from keras.layers import Bidirectional, Concatenate, Permute, Dot, LSTM, Multiply, Flatten
from keras.layers import RepeatVector, Activation
from keras.layers import BatchNormalization
from keras.models import Sequential, Model
from keras import backend as K
from keras.layers.embeddings import Embedding
from keras.models import load_model
from keras import optimizers as opt
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from sklearn.metrics import roc_auc_score

pd.options.display.max_rows = 10
pd.options.display.max_columns = 15
from IPython.display import display



import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os
import seaborn as sns
import tqdm
# import sentencepiece as spm

In [0]:
data = pd.read_csv('deceptive-opinion.csv')
# Keeping only the neccessary columns
data = data[['text','deceptive']]
data.head(1)
data = data.sample(frac=1)
data = data.sample(frac=1)
data = data.sample(frac=1)

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


data = data.replace(['truthful', 'deceptive'], [1, 0])
print(len(data[ data['deceptive'] == 1]))
print(len(data[ data['deceptive'] == 0]))

800
800


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/3, Total size: 810.60MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-lite/2"
module = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)
input_placeholder = tf.sparse_placeholder(tf.int64, shape=[None, None])
encodings = module(
    inputs=dict(
        values=input_placeholder.values,
        indices=input_placeholder.indices,
        dense_shape=input_placeholder.dense_shape))

with tf.Session() as sess:
    spm_path = sess.run(module(signature="spm_path"))

sp = spm.SentencePieceProcessor()
sp.Load(spm_path)
print("SentencePiece model loaded at {}.".format(spm_path))

SentencePiece model loaded at b'/tmp/tfhub_modules/539544f0a997d91c327c23285ea00c37588d92cc/assets/universal_encoder_8k_spm.model'.


In [0]:
def process_to_IDs_in_sparse_format(sp, sentences):
    # An utility method that processes sentences with the sentence piece processor
    # 'sp' and returns the results in tf.SparseTensor-similar format:
    # (values, indices, dense_shape)
    ids = [sp.EncodeAsIds(x) for x in sentences]
    max_len = max(len(x) for x in ids)
    dense_shape=(len(ids), max_len)
    values=[item for sublist in ids for item in sublist]
    indices=[[row,col] for row in range(len(ids)) for col in range(len(ids[row]))]
    return (values, indices, dense_shape)

In [0]:
def sentences_to_indices(X, max_len=512):
    
    m = X.shape[0]                                   # number of training examples
    vector = []
    X = X.values
#     print(X[0])
#     X_indices = np.zeros((m, max_len))
    with tf.Session() as session:
        for i in tqdm.tqdm(range(m)):                               # loop over training examples
            sentences = X[i].split('.')
    #             print(sentences)
            values, indices, dense_shape = process_to_IDs_in_sparse_format(sp, sentences)

            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_embeddings = session.run(encodings, feed_dict={input_placeholder.values: values,
                        input_placeholder.indices: indices,
                        input_placeholder.dense_shape: dense_shape})
            vector.append(np.array(message_embeddings).tolist())

    X_indices = np.array(vector)
    X_indices = np.squeeze(X_indices)
    return X_indices

In [0]:
# data=data[:100]
# data['text']
indices = sentences_to_indices(data['text'])
indices = indices.reshape(10, 512, 1)
indices.shape

In [0]:
import pickle

embeddings_file = "embeddings-{}.pickle".format(len(indices))
pickle.dump(indices, open(embeddings_file, 'wb'))

In [0]:
!cp embeddings-100.pickle drive/My\ Drive .

In [0]:
import pickle
indices = pickle.load(open('tweets_embeddings.pickle', 'rb'))

In [0]:
def euclidean_distance(inputs):
    assert len(inputs) == 2, \
        'Euclidean distance needs 2 inputs, %d given' % len(inputs)
    u, v = inputs
    return K.sqrt(K.sum((K.square(u - v)), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
    	signature="default", as_dict=True)["default"]


In [0]:
class xz():
    def __init__(self):
        self.x=1;
        self.z=0
def model_emb(obj, embed_size = 512):
    input_text = Input(shape=(1,), dtype=tf.string)
#     print(keras.backend.shape(input_text))
    
    
    embedding = Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
#     flat = Flatten()(embedding)
#     layer.output_shape

    flat = Reshape((512, obj.x))(embedding)
    obj.x = 32
    lstm = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.3))(flat)
    dense = Dense(128, activation='relu')(lstm)
    
    pred = Dense(2, activation='softmax')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='categorical_crossentropy', 
    optimizer=opt.Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])
    return model

In [0]:
def model_cdac(embed_size = 512):
    input_text = Input(shape=(1,), dtype=tf.string, name='input1')
#     print(keras.backend.shape(input_text))
    
    
    embedding = Lambda(UniversalEmbedding, output_shape=(embed_size,), name='embed1')(input_text)
    
    dense = Dense(512, activation='relu', name='dense1')(embedding)
    dense = Dense(256, activation='relu', name='dense2')(dense)
    dense = Dense(256, activation='relu', name='dense3')(dense)
#     dense = Dense(64, activation='relu', name='dense3')(dense)
#     dense = Dense(24, activation='relu', name='dense4')(dense)
    pred = Dense(2, activation='softmax', name='dense5')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='categorical_crossentropy', 
    optimizer=opt.Adam(lr=0.0001, decay=1e-5), metrics=['accuracy'])
    return model

In [0]:
Y_train = pd.get_dummies(data['deceptive']).values

X_train = data['text'].tolist()
X_train = np.array(X_train, dtype=object)[:, np.newaxis]
# model = load_model('model.h5')
# model = model_cdac()
# model.load_weights('model.h5')
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model = model_cdac()
    history = model.fit(X_train, 
            Y_train,
            validation_split=0.07,
            epochs=13,
            batch_size=32)
    model.save_weights('./model.h5')

Train on 1488 samples, validate on 112 samples
Epoch 1/13
1488/1488 [==============================] - 11s 7ms/step - loss: 0.6782 - acc: 0.6082 - val_loss: 0.6592 - val_acc: 0.6250
Epoch 2/13
1488/1488 [==============================] - 7s 5ms/step - loss: 0.6256 - acc: 0.7198 - val_loss: 0.5636 - val_acc: 0.8214
Epoch 3/13
1488/1488 [==============================] - 7s 5ms/step - loss: 0.5403 - acc: 0.7440 - val_loss: 0.4676 - val_acc: 0.7946
Epoch 4/13
1488/1488 [==============================] - 7s 5ms/step - loss: 0.4826 - acc: 0.7742 - val_loss: 0.4347 - val_acc: 0.8036
Epoch 5/13
1488/1488 [==============================] - 7s 5ms/step - loss: 0.4497 - acc: 0.7849 - val_loss: 0.4058 - val_acc: 0.8304
Epoch 6/13
1488/1488 [==============================] - 7s 5ms/step - loss: 0.4221 - acc: 0.8031 - val_loss: 0.4058 - val_acc: 0.8482
Epoch 7/13
1488/1488 [==============================] - 7s 5ms/step - loss: 0.4022 - acc: 0.8179 - val_loss: 0.3771 - val_acc: 0.8393
Epoch 8/13
148

In [0]:
new_text = ["In what year did the titanic sink ?", 
            "What is the highest peak in California ?", 
            "Who invented the light bulb ?"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
#     model.load_weights('./model.h5')  
    predicts = model.evaluate(X_train, Y_train, verbose=1)

# categories = df_train.label.cat.categories.tolist()
print(predicts)

  32/1600 [..............................] - ETA: 7:00

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7ff7cc129e10>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


1600/1600 [==============================] - 15s 10ms/step
[0.694139541387558, 0.470625]


In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

from google.colab import files
files.download('model.png')

In [0]:

def model_lstm_dec(vector_size):
    
    model = Sequential()

    model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same',
                     input_shape=(vector_size, 1)))
    model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
    model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=3))

    model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.3)))

    model.add(Dense(512, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(512, activation='sigmoid'))
    model.add(Dropout(0.25))

    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=opt.Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

    
    return model

In [0]:

model = model_lstm_dec(512)

Y_train = pd.get_dummies(data['deceptive']).values

nb_epoch = 1
history = model.fit(indices, Y_train, validation_split=0.05,
                    shuffle=True, batch_size=64, epochs=nb_epoch, verbose=1)   

Train on 9 samples, validate on 1 samples
Epoch 1/1
9/9 [==============================] - 16s 2s/step - loss: 0.6885 - acc: 0.5556 - val_loss: 0.4438 - val_acc: 1.0000


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 200)          80000200  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200, 512)          935936    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200, 512)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 70)                35910     
__________

In [0]:
X_train_q = data['text'].values
Y_train = pd.get_dummies(data['deceptive']).values

In [0]:
X_train_text = sentences_to_indices(X_train_q, word_to_index, 200) #X_train_q is list of sentences

In [0]:
nb_epoch = 1
history = model.fit(X_train_text, Y_train, validation_split=0.05,
                    shuffle=True, batch_size=64, epochs=nb_epoch, verbose=1)

Train on 1520 samples, validate on 80 samples
Epoch 1/1
1520/1520 [==============================] - 216s 142ms/step - loss: 0.4817 - acc: 0.7724 - val_loss: 0.5690 - val_acc: 0.7125


In [0]:
model.save_weights('dec_model.h5')

In [0]:
!mv dec_model.h5 drive/My\ Drive/

In [0]:
pred = model.predict([X_test], verbose=1)